In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cot_probing.patching import clean_run_with_cache, patched_run

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "hugging-quants/Meta-Llama-3.1-8B-BNB-NF4-BF16"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="cuda",
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt1 = "Today is a good day, I think I'll"
prompt2 = "Today is a bad day, I think I'll"
input_ids1 = tokenizer.encode(prompt1)
input_ids2 = tokenizer.encode(prompt2)
from cot_probing.vis import visualize_tokens_html

display(visualize_tokens_html(input_ids1, tokenizer))
display(visualize_tokens_html(input_ids2, tokenizer))

In [5]:
our_pos = slice(4, None)
our_layers = [5]
pos_by_layer = {l: [our_pos] for l in our_layers}

In [6]:
logits1, resid_by_pos_by_layer1 = clean_run_with_cache(model, input_ids1, pos_by_layer)
logits2, resid_by_pos_by_layer2 = clean_run_with_cache(model, input_ids2, pos_by_layer)
print(resid_by_pos_by_layer1[our_layers[0]][our_pos].shape)
print(resid_by_pos_by_layer2[our_layers[0]][our_pos].shape)
print(logits1.shape)
print(logits2.shape)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


torch.Size([7, 4096])
torch.Size([7, 4096])
torch.Size([128256])
torch.Size([128256])


In [7]:
logits_patched_1_to_2 = patched_run(model, input_ids2, resid_by_pos_by_layer1)
logits_patched_2_to_1 = patched_run(model, input_ids1, resid_by_pos_by_layer2)
print(logits_patched_1_to_2.shape)
print(logits_patched_2_to_1.shape)

torch.Size([128256])
torch.Size([128256])


In [8]:
# we're now returning only the last position, so it doesn't work anymore

# for seq in range(logits_patched_1_to_2.shape[0]):
#     allclose = torch.allclose(logits_patched_1_to_2[seq], logits2[seq])
#     print(f"seq {seq} allclose: {allclose}")
# for seq in range(logits_patched_2_to_1.shape[0]):
#     allclose = torch.allclose(logits_patched_2_to_1[seq], logits1[seq])
#     print(f"seq {seq} allclose: {allclose}")

In [9]:
topk_toks_1 = logits1.topk(5).indices.tolist()
topk_toks_2 = logits2.topk(5).indices.tolist()
display(visualize_tokens_html(topk_toks_1, tokenizer))
display(visualize_tokens_html(topk_toks_2, tokenizer))

In [10]:
top_toks_patched_1_to_2 = logits_patched_1_to_2.topk(5).indices.tolist()
top_toks_patched_2_to_1 = logits_patched_2_to_1.topk(5).indices.tolist()
display(visualize_tokens_html(top_toks_patched_1_to_2, tokenizer))
display(visualize_tokens_html(top_toks_patched_2_to_1, tokenizer))